In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_68801/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/10 18:08:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/10 18:08:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/10 18:08:43 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [8]:
# Sample DataFrame (replace this with your actual DataFrame)
data_address = [
    (1, 1),
    (2, 1),
    (3, 1),
    (4, 2),
    (5, 1),
    (6, 2),
    (7, 2),
    (8, 2),
]


columns = ["id", "num"]
df_person = spark.createDataFrame(data_address, columns)
# Register the DataFrame as a temporary view
df_person.createOrReplaceTempView("Logs")


# Sample DataFrame (replace this with your actual DataFrame)
data_address = [

    (1, 2, "New York City", "New York"),
    (2, 3, "Leetcode", "California"),

]
columns = ["addressId", "personId", "city", "state"]
df_address = spark.createDataFrame(data_address, columns)
# Register the DataFrame as a temporary view
df_address.createOrReplaceTempView("Address")

In [17]:
query = """
	select
		num
	from (select
			id,
			num,
			sum(
				case
					when num != previous_num then 1
					else 0
				end
			) over (order by id asc) as group_id
		from (select
			id,
			num,
			lag(num, 1, num) over (order by id asc) as previous_num
		from
			Logs) t)
	group by group_id, num
	having count(distinct id) >= 3
"""

In [18]:
spark.sql(query).show()

24/03/10 18:23:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/10 18:23:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/10 18:23:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/10 18:23:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/10 18:23:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/10 18:23:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/10 1

+---+
|num|
+---+
|  1|
|  2|
+---+

